# Importing Libraries


In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importing cleaned data

In [163]:
df = pd.read_parquet('D:/healthplusclinic/data/02-clean_data/cleaned_data.parquet')

In [164]:
df.head()

,appointment_id,patient_id,provider_id,appointment_date,appointment_time,lead_time_days,wait_time_minutes,is_no_show_0_1,age,insurance_type,specialty,provider_clinic_id,clinic_assignment,clinic_name,city,hours_start,hours_end
0,a0071090,p001027,pr00198,2024-01-07,12:15:00,45,33.0,0,82,private,oncology,c007,full-time,oncology clinic,chicago,10:00:00,19:00:00
1,a0074955,p000716,pr00002,2024-07-08,15:45:00,21,19.0,0,19,private,neurology,c006,full-time,neurology clinic,chicago,08:00:00,18:00:00
2,a0097350,p000507,pr00101,2024-01-16,10:15:00,31,18.0,0,30,private,pulmonology,c015,full-time,pulmonology clinic,houston,08:00:00,16:00:00
3,a0036456,p001164,pr00010,2024-05-19,15:45:00,12,29.0,0,18,public,pulmonology,c015,full-time,pulmonology clinic,houston,08:00:00,16:00:00
4,a0023637,p003837,pr00066,2025-03-26,15:15:00,7,19.0,0,56,public,orthopedics,c003,full-time,orthopedic clinic,houston,09:00:00,17:00:00


In [165]:
print(df.columns)

Index(['appointment_id', 'patient_id', 'provider_id', 'appointment_date',
       'appointment_time', 'lead_time_days', 'wait_time_minutes',
       'is_no_show_0_1', 'age', 'insurance_type', 'specialty',
       'provider_clinic_id', 'clinic_assignment', 'clinic_name', 'city',
       'hours_start', 'hours_end'],
      dtype='str')


In [166]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   appointment_id      120000 non-null  str           
 1   patient_id          120000 non-null  str           
 2   provider_id         120000 non-null  str           
 3   appointment_date    120000 non-null  datetime64[us]
 4   appointment_time    120000 non-null  object        
 5   lead_time_days      120000 non-null  int64         
 6   wait_time_minutes   120000 non-null  float64       
 7   is_no_show_0_1      120000 non-null  int64         
 8   age                 120000 non-null  int64         
 9   insurance_type      120000 non-null  str           
 10  specialty           120000 non-null  str           
 11  provider_clinic_id  120000 non-null  str           
 12  clinic_assignment   120000 non-null  str           
 13  clinic_name         120000 non-null  str

# Feature Engineering


#### Creating new column whethers the appointment was booked on weekend or weekdays

In [167]:
df['days_week'] = df['appointment_date'].dt.isocalendar().day #parsing day of week from appointment_date column

In [168]:
df.head()

,appointment_id,patient_id,provider_id,appointment_date,appointment_time,lead_time_days,wait_time_minutes,is_no_show_0_1,age,insurance_type,specialty,provider_clinic_id,clinic_assignment,clinic_name,city,hours_start,hours_end,days_week
0,a0071090,p001027,pr00198,2024-01-07,12:15:00,45,33.0,0,82,private,oncology,c007,full-time,oncology clinic,chicago,10:00:00,19:00:00,7
1,a0074955,p000716,pr00002,2024-07-08,15:45:00,21,19.0,0,19,private,neurology,c006,full-time,neurology clinic,chicago,08:00:00,18:00:00,1
2,a0097350,p000507,pr00101,2024-01-16,10:15:00,31,18.0,0,30,private,pulmonology,c015,full-time,pulmonology clinic,houston,08:00:00,16:00:00,2
3,a0036456,p001164,pr00010,2024-05-19,15:45:00,12,29.0,0,18,public,pulmonology,c015,full-time,pulmonology clinic,houston,08:00:00,16:00:00,7
4,a0023637,p003837,pr00066,2025-03-26,15:15:00,7,19.0,0,56,public,orthopedics,c003,full-time,orthopedic clinic,houston,09:00:00,17:00:00,3


In [169]:
# creating new column by checking whether the appoinment day is weekend or not
df['is_weekend'] = df['days_week'].apply(lambda x: True if (x==6 or x==7) else False)

In [170]:
# converting to hours from appointment time and creating new column
df['hour'] = df['appointment_time'].apply(lambda x: x.hour)

In [171]:
# creating new column from hours by binning
bins = [0,12,17,24]
labels = ['Morning', 'Afternoon','Evening']
df['time_of_day'] = pd.cut(df['hour'], bins=bins, labels=labels, right=False)


In [172]:
df.drop(columns='hour',inplace=True)

In [173]:
# creating new columns for how long time clinic opens
# converting hours_start and hours_end to string type, both columns were previoulsy converted into datetime object
# It is converted as timedelta expects str types
df['open_hours'] = pd.to_timedelta(df['hours_end'].astype(str)) - pd.to_timedelta(df['hours_start'].astype(str))
df['open_hours'] = (df['open_hours'].dt.total_seconds()/3600).astype(int)

In [174]:
# deleting both open hours and end hours column as we don't need that one
df.drop(['hours_start','hours_end'], axis=1, inplace=True)

In [175]:
# defining age group grom age of pateints
print('Max Age: ',df['age'].max())
print('Min Age: ',df['age'].min())

Max Age:  90
Min Age:  0


In [176]:
bins = [0,3,17,31,45,np.inf]
labels = ['babies','childern','young','middle_age','old']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right = False)

In [177]:
# creating new column how many times patient visited clinic until last time
df = df.sort_values(['patient_id','appointment_date'], ascending=True)
df['total_appointments'] = df.groupby('patient_id').cumcount()
df.reset_index(level=0, drop=True, inplace=True)

In [178]:
# creating column with no show rate history of pateints
df['patient_no_show_rate'] = df.groupby('patient_id')['is_no_show_0_1'].transform(lambda x: x.shift().expanding().mean())
df['patient_no_show_rate']=df['patient_no_show_rate'].fillna(0)

In [179]:
# creating new column whether the booking is for weekdays or not
df['is_weekdays'] = df['days_week'].apply(lambda x: True if (x>0 and x<6) else False)

In [180]:
df.head(5)

,appointment_id,patient_id,provider_id,appointment_date,appointment_time,lead_time_days,wait_time_minutes,is_no_show_0_1,age,insurance_type,...,clinic_name,city,days_week,is_weekend,time_of_day,open_hours,age_group,total_appointments,patient_no_show_rate,is_weekdays
0,a0022327,p000001,pr00139,2023-01-19,12:15:00,2,19.0,0,63,private,...,ophthalmology clinic,houston,4,False,Afternoon,10,old,0,0.0,True
1,a0084759,p000001,pr00089,2023-01-20,14:30:00,0,21.0,0,63,private,...,women's health clinic,atlanta,5,False,Afternoon,8,old,1,0.0,True
2,a0093526,p000001,pr00071,2023-04-17,18:15:00,9,33.0,0,63,private,...,dental clinic,new york,1,False,Evening,9,old,2,0.0,True
3,a0071212,p000001,pr00117,2023-05-06,07:00:00,15,21.0,0,63,private,...,ophthalmology clinic,houston,6,True,Morning,10,old,3,0.0,False
4,a0002445,p000001,pr00181,2023-05-08,18:45:00,8,54.0,0,63,private,...,orthopedic clinic,houston,1,False,Evening,8,old,4,0.0,True


In [181]:

dict = df['clinic_name'].value_counts().to_dict()
print(dict)
df['clinic_count'] = df['clinic_name'].map(dict)

{'orthopedic clinic': 14254, 'dental clinic': 12305, 'oncology clinic': 11891, 'pulmonology clinic': 11549, 'urology clinic': 11548, 'psychiatry clinic': 10954, 'dermatology clinic': 9239, 'cardiology clinic': 8084, 'gastroenterology clinic': 7713, 'ophthalmology clinic': 5470, 'ent clinic': 5030, "women's health clinic": 3884, 'primary care clinic': 3777, 'neurology clinic': 2189, 'endocrinology clinic': 2113}


In [182]:
bins = [0,5000,10000,np.inf]
labels = ['low_volume_clinic', 'medium_volume_clinic','high_volume_clinic']
df['patient_clinic_frequency_visit'] = pd.cut(df['clinic_count'], bins=bins, labels=labels, right=False)

In [183]:
df.head()

,appointment_id,patient_id,provider_id,appointment_date,appointment_time,lead_time_days,wait_time_minutes,is_no_show_0_1,age,insurance_type,...,days_week,is_weekend,time_of_day,open_hours,age_group,total_appointments,patient_no_show_rate,is_weekdays,clinic_count,patient_clinic_frequency_visit
0,a0022327,p000001,pr00139,2023-01-19,12:15:00,2,19.0,0,63,private,...,4,False,Afternoon,10,old,0,0.0,True,5470,medium_volume_clinic
1,a0084759,p000001,pr00089,2023-01-20,14:30:00,0,21.0,0,63,private,...,5,False,Afternoon,8,old,1,0.0,True,3884,low_volume_clinic
2,a0093526,p000001,pr00071,2023-04-17,18:15:00,9,33.0,0,63,private,...,1,False,Evening,9,old,2,0.0,True,12305,high_volume_clinic
3,a0071212,p000001,pr00117,2023-05-06,07:00:00,15,21.0,0,63,private,...,6,True,Morning,10,old,3,0.0,False,5470,medium_volume_clinic
4,a0002445,p000001,pr00181,2023-05-08,18:45:00,8,54.0,0,63,private,...,1,False,Evening,8,old,4,0.0,True,14254,high_volume_clinic


In [184]:
# dropping columns that are not needed for modeling
cols = ['appointment_id','patient_id','provider_id','appointment_date','appointment_time','days_week','provider_clinic_id','specialty','clinic_name','clinic_count']
df = df.drop(columns=cols)

In [185]:
df.head()

,lead_time_days,wait_time_minutes,is_no_show_0_1,age,insurance_type,clinic_assignment,city,is_weekend,time_of_day,open_hours,age_group,total_appointments,patient_no_show_rate,is_weekdays,patient_clinic_frequency_visit
0,2,19.0,0,63,private,full-time,houston,False,Afternoon,10,old,0,0.0,True,medium_volume_clinic
1,0,21.0,0,63,private,full-time,atlanta,False,Afternoon,8,old,1,0.0,True,low_volume_clinic
2,9,33.0,0,63,private,full-time,new york,False,Evening,9,old,2,0.0,True,high_volume_clinic
3,15,21.0,0,63,private,full-time,houston,True,Morning,10,old,3,0.0,False,medium_volume_clinic
4,8,54.0,0,63,private,full-time,houston,False,Evening,8,old,4,0.0,True,high_volume_clinic


In [186]:
df.to_csv('D:/healthplusclinic/data/03-feature_eng_data/engineered_data.csv')
df.to_parquet('D:/healthplusclinic/data/03-feature_eng_data/engineered_data.parquet')